In [ ]:
###############################################################################################
# Working prototype of Modulation Excitation Spectroscopy (MES) data processing protocol,     #
# Written by Ryan Hagmann of the Hermans Research Group, UW-Madison, May 2023.                #
# Turn functions on and off by commenting with "#" before lines                               #
###############################################################################################

In [ ]:
# Import necessary libraries.

import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
import scipy.fft    # Uses built in Discrete F.T. algorithms from Scipy
from matplotlib import cm
from cycler import cycler
from scipy import stats
from math import factorial

In [ ]:
# Some useful functions, FFT/DFT replaced with Numpy implementation.

def phase_angle(complex_value):
    """Finds an angle in the complex plane (here the phase angle) for a given complex number."""
    angle = -np.angle(complex_value, deg = True)    # deg = False returns radians
    if angle < 0:
        angle += 360
    return angle

phi = 1    # Rotation angle in degrees

def rotate(complex_value, theta = (phi*np.pi/180)):
    """Rotate complex values counterclockwise in the imaginary plane by theta, preserving magnitude in output."""
    vector = np.array([[np.real(complex_value)], 
                       [np.imag(complex_value)]])    # Convert a + ib to vector [[a], [b]]
    operator = np.array([[np.cos(theta), -np.sin(theta)],    # Create rotation matrix
                         [np.sin(theta), np.cos(theta)]])
    new_vector = operator@vector    # Matrix multiplication for a rotated vector
    return new_vector[0][0].item()+new_vector[1][0].item()*1j

# Converting between y-axis units for reflectance measurements. Probably just stick with K.M. for qualitative analysis.

def KM(R):
    """Kubelka-Munk transform"""
    return np.power(1-R, 2)/(2*R)

def inv_KM(KM):
    """Undo the Kubelka-Munk transform"""
    return (1 + KM - np.sqrt(np.power(KM, 2) + 2*KM))

def logR(R):
    """log(R) is used as an alernative to Kubelka-Munk."""
    return np.log10(R)

def log_inv_R(R):
    # return np.log10(np.divide(1, R))
    return -np.log10(R)    # This is equivalent to the line above

In [ ]:
# Borrowed Savitzky-Golay algorithm

def savitzky_golay(y, window_size = 50, order = 2, deriv = 0, rate = 1):
    """Smooth (and optionally differentiate) data with a Savitzky-Golay filter. The Savitzky-Golay filter removes high frequency noise from data.
    It has the advantage of preserving the original shape and features of the signal better than other types of filtering approaches, such as 
    moving averages techniques.
    Parameters
    ----------
    y : array_like, shape (N,)
        the values of the time history of the signal.
        window_size : int
        the length of the window. Must be an odd integer number.
        order : int
        the order of the polynomial used in the filtering.
        Must be less then `window_size` - 1.
    deriv: int
    the order of the derivative to compute (default = 0 means only smoothing)
    Returns
    -------
    ys : ndarray, shape (N)
    the smoothed signal (or it's n-th derivative).
    Notes
    -----
    The Savitzky-Golay is a type of low-pass filter, particularly suited for smoothing noisy data. The main idea behind this approach is to make 
    for each point a least-square fit with a polynomial of high order over a odd-sized window centered at the point.
    Examples
    --------
    t = np.linspace(-4, 4, 500)
    y = np.exp( -t**2 ) + np.random.normal(0, 0.05, t.shape)
    ysg = savitzky_golay(y, window_size=31, order=4)
    import matplotlib.pyplot as plt
    plt.plot(t, y, label='Noisy signal')
    plt.plot(t, np.exp(-t**2), 'k', lw=1.5, label='Original signal')
    plt.plot(t, ysg, 'r', label='Filtered signal')
    plt.legend()
    plt.show()
    References
    ----------
    [1] A. Savitzky, M. J. E. Golay, Smoothing and Differentiation of Data by Simplified Least Squares Procedures. Analytical Chemistry, 1964, 36 
       (8), pp 1627-1639.
    [2] Numerical Recipes 3rd Edition: The Art of Scientific Computing W.H. Press, S.A. Teukolsky, W.T. Vetterling, B.P. Flannery Cambridge 
    University Press ISBN-13: 9780521880688
    """
    # import numpy as np
    # from math import factorial

    try:
        window_size = np.abs(int(window_size))
        order = np.abs(int(order))
    except ValueError:
        raise ValueError("window_size and order have to be of type int")
        if window_size % 2 != 1 or window_size < 1:
            raise TypeError("window_size size must be a positive odd number")
        if window_size < order + 2:
            raise TypeError("window_size is too small for the polynomials order")
    order_range = range(order+1)
    half_window = (window_size -1) // 2
    # precompute coefficients
    b = np.mat([[k**i for i in order_range] for k in range(-half_window, half_window+1)])
    m = np.linalg.pinv(b).A[deriv] * rate**deriv * factorial(deriv)
    # pad the signal at the extremes with
    # values taken from the signal itself
    firstvals = y[0] - np.abs( y[1:half_window+1][::-1] - y[0] )
    lastvals = y[-1] + np.abs(y[-half_window-1:-1][::-1] - y[-1])
    y = np.concatenate((firstvals, y, lastvals))
    return np.convolve( m[::-1], y, mode='valid')

In [ ]:
# Reads data from data point table *.dpt files extracted from OPUS Rapid Scan protocol using pandas.
# Can be modified to read any tabular data format: excel, csv, etc.

path = r"C:\Users\ryanh\OneDrive - UW-Madison\Hermans_Group\ME-DRIFTS\High_pressure\2023-04-25-68Ga-KBr-FI-35bar\MES_260C"    # Set working directory, called again to save processed data
file = r"\40H2-20CO2-MES-4-32-32.0.dpt"    # The specifc path extension of the file
file_path = path + file    # Combine for full path 
print(file_path)    # To ensure correct path

dpt = pd.read_csv(file_path, sep = ",", header = None, delim_whitespace = False)    # Import data, *.dpt files from OPUS are in csv format
dim = dpt.shape

print("Number of wavenumbers sampled over wavenumber range: " + str(dim[0]))    # Check dimensions of imported data to ensure it matches expected number of scans, etc.
print("Number of scans: " + str(dim[1] - 1))

# The data frame is imported like so: wavenumbers in the first column, scans over that spectrum in successive columns.
# Data is organized by wavenumber, each row is a subsequent scan

header = []
for wavenumber in dpt.iloc[0: , 0]:    # Round to nearest wavenumber for convenience
    header.append(str(round(wavenumber)))

data = dpt.T[1:]
data.columns = header
data.head()

In [ ]:
# Convert data into log(1/R) scale

# data = log_inv_R(inv_KM(data))

In [ ]:
data_stats = data.describe()
data_stats

In [ ]:
# Optional
# Remove outliers, any values with z > 3, usually unnecessary at low pressure

cleaned_data = data[(np.abs(scipy.stats.zscore(data)) < 3).all(axis=1)]
data = cleaned_data    # Use data without outliers for subsequent cells
data.head()

In [ ]:
# Optional
# Apply Savitsky-Golay filter to cleaned data, usually unnecessary at low pressure
# As always, be very careful with smoothing your data

smoothed_data = pd.DataFrame(data.apply(savitzky_golay, axis = 0, raw = True))#, columns = data.columns)
data = smoothed_data    # Use Savitsky-Golay smoothed data for subsequent cells
data.head()

In [ ]:
# Show a few key frequencies over the course of the experiment.
# Make a figure with an oscillating wavenumber

start = 0 # initial scan number 
end = dim[1] - 1 # final scan number

fig, ax = plt.subplots(1, 1, figsize=(15, 5))
ax.plot(data["1988"]/np.max(data["1988"]), "g", label = "Bridging C=O: 1988 $cm^{-1}$")    # Ga-H
ax.plot(data["2358"]/np.max(data["2358"]), "r", label = "CO$_2$: 2358 $cm^{-1}$")    # CO2
ax.plot(data["1581"]/np.max(data["1581"]), "b", label = "Formate: 1581 $cm^{-1}$")    # Carbonates/formate?
#ax.plot(wavenumbers, dpt.iloc[:,dim[1]-1], "r", label="Last spectrum")
plt.xlabel("Seconds")
plt.ylabel("Normalized Signal")
plt.yticks([])
plt.title("Time Domain Data")
ax.set(
    xlim = (start, end),
    ylim = (0, 1)
    #xticks = np.arange(4000, 700, -500),
    )
ax.legend(loc="best")
# plt.savefig(path + r"\time_domain_signal.png", dpi = 300)
plt.show()

In [ ]:
# Average desired MES periods.
# Using above periods, decide which periods to average for MES, they should be all similar to satisfy a quasi-steady-state.

n = 3 # Remove first n periods
N = 10 # Last period to consider, 
scans_per_spectrum = 120
i = n

assert 0 < n < 10

spectrum = (i * 120)   # i = n
# avgd_period = smoothed_data.iloc[spectrum: spectrum + scans_per_spectrum, : ].reset_index().iloc[ : , 1: ]
avgd_period = data.iloc[spectrum: spectrum + scans_per_spectrum, : ].reset_index().iloc[ : , 1: ]

# Average remaining periods

while i < (N - 1):    # We usually use 10 periods, usually discard the first three
    i += 1    # next period
    subsequent_spectrum = (i * scans_per_spectrum)
    # sum_period = avgd_period.add(smoothed_data.iloc[subsequent_spectrum: subsequent_spectrum + scans_per_spectrum, : ].reset_index()).iloc[ : , 1: ])
    sum_period = avgd_period.add(data.iloc[subsequent_spectrum: subsequent_spectrum + scans_per_spectrum, : ].reset_index().iloc[ : , 1: ])
    avgd_period = sum_period
    # print(avgd_period.head(1))

# avgd_period /= (N - n)    # Normalize
avgd_period.tail(5)

In [ ]:
# Set mean intensity of each row (set of scans for each wavenumber) as 0.

centered_period = avgd_period.sub(avgd_period.mean()).reset_index(drop = True)    # As in doc string

# Display averaged and centered period with normalized signals.

fig, ax = plt.subplots(1, 1, figsize=(7.5, 5))
scans = np.arange(1, 121)
ax.plot(centered_period["1988"]/np.max(centered_period["1988"]), "g", label = "1988 $cm^{-1}$")
ax.plot(centered_period["2358"]/np.max(centered_period["2358"]), "r", label = "2358 $cm^{-1}$")
ax.plot(centered_period["1581"]/np.max(centered_period["1581"]), "b", label = "1581 $cm^{-1}$")
ax.plot(np.real(np.exp(-1j*2*np.pi*scans/120)), "k", label = "rFFT")
ax.plot(np.imag(np.exp(-1j*2*np.pi*scans/120)), "m", label = "iFFT")
plt.xlabel("Scans over time")
plt.ylabel("Signal")
plt.title("Selected Wavenumbers")
ax.set(
    #xlim = (start, end),
    #ylim = (0, 2)
    #xticks = np.arange(4000, 700, -500),
    )
ax.legend(loc="best")
plt.show()

In [ ]:
centered_period.head()

In [ ]:
# Perform a discrete Fourier transform over the scans in each column/wavenumber.

centered_array = centered_period.T.to_numpy()

dft_array = scipy.fft.fft(centered_array)
dft = pd.DataFrame(dft_array.T, columns = centered_period.columns)
dft.head()

In [ ]:
# Look at frequencies to ascertain waveform: sinusoidal, triangular, square...

# For more information see: 
# Srinivasan, P. D.; Patil, B. S.; Zhu, H.; Bravo-Suárez, J. J. Application of Modulation Excitation-Phase 
# Sensitive Detection-DRIFTS for in Situ /Operando Characterization of Heterogeneous Catalysts. React. Chem. Eng. 
# 2019, 4 (5), 862–883. https://doi.org/10.1039/C9RE00011A.

freq_range = 10

plt.figure(figsize = (12, 6))

plt.subplot(131)
plt.stem(np.arange(-len(dft["2358"])/2, len(dft["2358"])/2),
         np.abs(scipy.fft.fftshift(dft["2358"])), 
         linefmt='r',
         markerfmt=" ", 
         basefmt="-r")
plt.xlabel('Freq (Hz)')
plt.ylabel('FFT Amplitude |X(freq)|')
plt.xlim(-freq_range, freq_range)
plt.title("2358 $cm^{-1}$")

plt.subplot(132)
plt.stem(np.arange(-len(dft["1988"])/2, len(dft["1988"])/2),
         np.abs(scipy.fft.fftshift(dft["1988"])), 
         linefmt='g',
         markerfmt=" ", 
         basefmt="-g")
plt.xlabel('Freq (Hz)')
plt.ylabel('FFT Amplitude |X(freq)|')
plt.xlim(-freq_range, freq_range)
plt.title("1988 $cm^{-1}$")

plt.subplot(133)
plt.stem(np.arange(-len(dft["1581"])/2, len(dft["1581"])/2),
         np.abs(scipy.fft.fftshift(dft["1581"])), 
         linefmt='b',
         markerfmt=" ", 
         basefmt="-b")
plt.xlabel('Freq (Hz)')
plt.ylabel('FFT Amplitude |X(freq)|')
plt.xlim(-freq_range, freq_range)
plt.title("1581 $cm^{-1}$")

plt.tight_layout()
# plt.savefig(path + r"\Freq-domain.png", dpi = 300)
plt.show()

In [ ]:
# Revert column titles to experimental wavenumbers.

dft.columns = dpt.iloc[0: , 0]

# Find peak phase angles for each wavenumber.

angles = dft.applymap(phase_angle)    # Returns all phase angles in radians
# angles

# Rotate complex values for each wavenumber.
# Create a dataframe of spectra for all specified phase angles.

i = 1 # Index for iteration
phase_angle_array = pd.DataFrame(dft.iloc[1, : ]).T.to_numpy()    # Initiate numpy array

# Create a rotated set of Fourier transform outputs, so that phase angles can be generated by taking only their real components

while i < 360:    # This should be vectorized somehow for better performance
    new_row = []
    for item in phase_angle_array[-1]:    # Iteratively rotates the prior row to create the next row
        new_row.append(rotate(item))
    new = np.vstack((phase_angle_array, new_row))
    phase_angle_array = new
    i += 1
    
all_phase_angles = pd.DataFrame(data = phase_angle_array, columns = dft.columns)
# all_phase_angles.head()

# Revert column titles to experimental wavenumbers.

angles.columns = dpt.iloc[0: , 0]
# angles

In [ ]:
# Phase angle by wavenumber plot.

fig, ax = plt.subplots(1, 1, figsize=(25, 5))
plt.plot(angles.columns, angles.iloc[1, :])
ax.set(
    xlim = (4000, 800),
    ylim = (0, 360),
    xticks = np.arange(4000, 750, -100),
    yticks = np.arange(0, 370, 20)
)
plt.title("Phase Angles")
plt.xlabel("Wavenumber $cm^{-1}$")
plt.ylabel("Angle")
# fig.savefig(path + "\\phase_angles.jpg")
plt.show()

In [ ]:
# Take the real components.

real_components = all_phase_angles.applymap(np.real)

# Revert column titles to experimental wavenumbers.

real_components.columns = dpt.iloc[0: , 0]
# real_components

In [ ]:
# Correct background by a constant offset.
# This is dubious and makes many assumptions, consider carefully whether this is necessary and valid.

# Pick a range with no spectral signal, ~100 cm-1
high = 2700    # Maximum wavenumber
low = 2600    # Minimum wavenumber

in_range = []
for value in real_components.columns:
    if (value < high) and (value > low):
        in_range.append(value)
        
# real_components[in_range]    # The data within selected wavenumber range
means = real_components[in_range].mean(axis = 1)    # Average each row
real_components = real_components.subtract(means, axis = 0)    # Subract the mean of the selected range ffrom each row

In [ ]:
# Plot real components.

interval = 30    # In degrees, between phase angles displayed
phase_angles = np.arange(0, 360, interval)    # Which angles to plot using above interval

N = int(360 / interval)    # Match number of angles to number of colors
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", 
                                             # plt.cm.coolwarm(np.linspace(0,1,N))    # Redefine color list to coolwarm scheme
                                             # plt.cm.viridis(np.linspace(0,1,N))    # Redefine color list to viridis scheme
                                             # plt.cm.gnuplot2(np.linspace(0,1,N))    # Redefine color list gnuplot2 scheme
                                             plt.cm.rainbow(np.linspace(0,1,N))    # Redefine color list rainbow scheme
                                            )

y_scale = 0.25

fig, ax = plt.subplots(1, 1, figsize=(25, 10))
for angle in phase_angles:    # No more copying line by line to add angles
    ax.plot(real_components.columns, real_components.iloc[angle, :], label = str(angle))
ax.set(
    xlim = (4000, 800),
    ylim = (-y_scale, y_scale),
    xticks = np.arange(4000, 750, -100),
    #yticks = np.arange(0, 370, 20)
)
plt.title("Phase Angles")
plt.xlabel("Wavenumber $cm^{-1}$")
plt.ylabel("Kubelka-Munk")
plt.legend(loc = "upper left")
# fig.savefig(path + "\\spectrum_by_phase_angles.jpg")    # To save figure, uncomment
plt.show()

In [ ]:
# Generate phase angles for 2nd and 3rd harmonics.
# Higher harmonics may be useful for considering shorter periods or looking at more quickly forming species.

# 2nd, corresponding to period of 1/2 the actual period
i = 1    # Index for iteration
phase_angle_array2 = pd.DataFrame(dft.iloc[2, : ]).T.to_numpy()    # Initiate numpy array

while i < 360:
    new_row = []
    for item in phase_angle_array2[-1]:
        new_row.append(rotate(item))
    new = np.vstack((phase_angle_array2, new_row))
    phase_angle_array2 = new
    i += 1
    
all_phase_angles2 = pd.DataFrame(data = phase_angle_array2, columns = dft.columns)
# all_phase_angles.head()

# Take the real components.

real_components2 = all_phase_angles2.applymap(np.real)

# Revert column titles to experimental wavenumbers.

real_components2.columns = dpt.iloc[0: , 0]

# 3rd, corresponding to period of 1/3 the actual period
i = 1     # Index for iteration
phase_angle_array3 = pd.DataFrame(dft.iloc[3, : ]).T.to_numpy()    # Initiate numpy array

while i < 360:
    new_row = []
    for item in phase_angle_array3[-1]:
        new_row.append(rotate(item))
    new = np.vstack((phase_angle_array3, new_row))
    phase_angle_array3 = new
    i += 1
    
all_phase_angles3 = pd.DataFrame(data = phase_angle_array3, columns = dft.columns)
# all_phase_angles.head()

# Take the real components.

real_components3 = all_phase_angles3.applymap(np.real)

# Revert column titles to experimental wavenumbers.

real_components3.columns = dpt.iloc[0: , 0]

In [ ]:
# Plot real components.

interval = 30    # In degrees
phase_angles = np.arange(0, 360, interval)    # Which angles to plot
# phase_angles = [70, 160, 250, 270, 315, 340]

N = int(360 / interval)    # Match number of angles to number of colors
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", 
                                             # plt.cm.coolwarm(np.linspace(0,1,N))    # Redefine color list to coolwarm scheme
                                             # plt.cm.viridis(np.linspace(0,1,N))    # Redefine color list to viridis scheme
                                             # plt.cm.gnuplot2(np.linspace(0,1,N))    # Redefine color list gnuplot2 scheme
                                             plt.cm.rainbow(np.linspace(0,1,N))    # Redefine color list rainbow scheme
                                            )
x_upper_lim = 4000
x_lower_lim = 800
tick_step = 100
x_ticks = np.arange(x_upper_lim, x_lower_lim - tick_step, -tick_step)
y_scale = 0.2

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(25, 15))

# period = T
for angle in phase_angles:    # No more copying line by line to add angles
    ax1.plot(real_components.columns, real_components.iloc[angle, :], label = str(angle))
ax1.set(
    xlim = (x_upper_lim, x_lower_lim),
    ylim = (-y_scale, y_scale),
    xticks = x_ticks,
    ylabel = "Kubelka-Munk",
    title = "Fundamental Phase Angles"
)

# Period = T/2
for angle in phase_angles:    # No more copying line by line to add angles
    ax2.plot(real_components2.columns, real_components2.iloc[angle, :], label = str(angle))
ax2.set(
    xlim = (x_upper_lim, x_lower_lim),
    ylim = (-y_scale, y_scale),
    xticks = x_ticks,
    ylabel = "Kubelka-Munk",
    title = "Second Harmonic (T/2) Phase Angles"
)

# Period = T/3
for angle in phase_angles:    # No more copying line by line to add angles
    ax3.plot(real_components3.columns, real_components3.iloc[angle, :], label = str(angle))
ax3.set(
    xlim = (x_upper_lim, x_lower_lim),
    ylim = (-y_scale, y_scale),
    xticks = x_ticks,
    ylabel = "Kubelka-Munk",
    title = "Third Harmonic (T/3) Phase Angles"
)

plt.xlabel("Wavenumber $cm^{-1}$")
ax1.legend(loc = "upper left")
ax2.legend(loc = "upper left")
ax3.legend(loc = "upper left")
fig.savefig(path + "\\spectrum_by_phase_angles_gratuitous_harmonics.jpg")
plt.show()

In [ ]:
# Generate phase angles for 4th, 5th, and 6th harmonics.

# 4th, corresponding to period of 1/4 the actual period
i = 1     # Index for iteration
phase_angle_array4 = pd.DataFrame(dft.iloc[4, : ]).T.to_numpy()    # Initiate numpy array

while i < 360:
    new_row = []
    for item in phase_angle_array4[-1]:
        new_row.append(rotate(item))
    new = np.vstack((phase_angle_array4, new_row))
    phase_angle_array4 = new
    i += 1
    
all_phase_angles4 = pd.DataFrame(data = phase_angle_array4, columns = dft.columns)

# Take the real components.

real_components4 = all_phase_angles4.applymap(np.real)

# Revert column titles to experimental wavenumbers.

real_components4.columns = dpt.iloc[0: , 0]

# 5th, corresponding to period of 1/5 the actual period
i = 1     # Index for iteration
phase_angle_array5 = pd.DataFrame(dft.iloc[5, : ]).T.to_numpy()    # Initiate numpy array

while i < 360:
    new_row = []
    for item in phase_angle_array5[-1]:
        new_row.append(rotate(item))
    new = np.vstack((phase_angle_array5, new_row))
    phase_angle_array5 = new
    i += 1
    
all_phase_angles5 = pd.DataFrame(data = phase_angle_array5, columns = dft.columns)

# Take the real components.

real_components5 = all_phase_angles5.applymap(np.real)

# Revert column titles to experimental wavenumbers.

real_components5.columns = dpt.iloc[0: , 0]

# 6th, corresponding to period of 1/5 the actual period
i = 1    # Index for iteration
phase_angle_array6 = pd.DataFrame(dft.iloc[6, : ]).T.to_numpy()    # Initiate numpy array

while i < 360:
    new_row = []
    for item in phase_angle_array6[-1]:
        new_row.append(rotate(item))
    new = np.vstack((phase_angle_array6, new_row))
    phase_angle_array6 = new
    i += 1
    
all_phase_angles6 = pd.DataFrame(data = phase_angle_array6, columns = dft.columns)

# Take the real components.

real_components6 = all_phase_angles6.applymap(np.real)

# Revert column titles to experimental wavenumbers.

real_components6.columns = dpt.iloc[0: , 0]

In [ ]:
# Plot real components.

interval = 30    # In degrees
phase_angles = np.arange(0, 360, interval)    # Which angles to plot
# phase_angles = [70, 160, 250, 270, 315, 340]

N = int(360 / interval)    # Match number of angles to number of colors
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", 
                                             # plt.cm.coolwarm(np.linspace(0,1,N))    # Redefine color list to coolwarm scheme
                                             # plt.cm.viridis(np.linspace(0,1,N))    # Redefine color list to viridis scheme
                                             # plt.cm.gnuplot2(np.linspace(0,1,N))    # Redefine color list gnuplot2 scheme
                                             plt.cm.rainbow(np.linspace(0,1,N))    # Redefine color list rainbow scheme
                                            )
x_upper_lim = 4000
x_lower_lim = 800
tick_step = 100
x_ticks = np.arange(x_upper_lim, x_lower_lim - tick_step, -tick_step)
y_scale = 0.05

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(25, 15))

# period = T/4
for angle in phase_angles:    # No more copying line by line to add angles
    ax1.plot(real_components4.columns, real_components4.iloc[angle, :], label = str(angle))
ax1.set(
    xlim = (x_upper_lim, x_lower_lim),
    ylim = (-y_scale, y_scale),
    xticks = x_ticks,
    ylabel = "Kubelka-Munk",
    title = "Fourth Harmonic (T/4) Phase Angles"
)

# Period = T/5
for angle in phase_angles:    # No more copying line by line to add angles
    ax2.plot(real_components5.columns, real_components5.iloc[angle, :], label = str(angle))
ax2.set(
    xlim = (x_upper_lim, x_lower_lim),
    ylim = (-y_scale, y_scale),
    xticks = x_ticks,
    ylabel = "Kubelka-Munk",
    title = "Fifth Harmonic (T/5) Phase Angles"
)

# Period = T/6
for angle in phase_angles:    # No more copying line by line to add angles
    ax3.plot(real_components6.columns, real_components6.iloc[angle, :], label = str(angle))
ax3.set(
    xlim = (x_upper_lim, x_lower_lim),
    ylim = (-y_scale, y_scale),
    xticks = x_ticks,
    ylabel = "Kubelka-Munk",
    title = "Sixth Harmonic (T/6) Phase Angles"
)

plt.xlabel("Wavenumber $cm^{-1}$")
ax1.legend(loc = "upper left")
ax2.legend(loc = "upper left")
ax3.legend(loc = "upper left")
# fig.savefig(path + "\\spectrum_by_phase_angles_gratuitous_higher_harmonics.jpg")    # Uncomment to save
plt.show()

In [ ]:
# Phase angle by wavenumber plot for selected harmonic.

fig, ax = plt.subplots(1, 1, figsize=(25, 5))
plt.plot(angles.columns, angles.iloc[6, :])
ax.set(
    xlim = (4000, 800),
    ylim = (0, 360),
    xticks = np.arange(4000, 750, -100),
    yticks = np.arange(0, 370, 20)
)
plt.title("Phase Angles for 6th Harmonic")
plt.xlabel("Wavenumber $cm^{-1}$")
plt.ylabel("Angle")
# fig.savefig(path + "\\phase_angles6.jpg")    # Uncomment to save
plt.show()

In [ ]:
# Save numerical results for fundamental frequency.

# Fourier transformed output and phase angles

output_and_angles = pd.concat([dft.iloc[1, : ], angles.iloc[1, : ]], axis = 1)
output_and_angles.columns = ["Discrete Fourier Transform", "Phase Angle"]
save_to_excel_name = path + "\\FT_output_and_angles.xlsx"
output_and_angles.to_excel(save_to_excel_name)

# Save real components by phase angle data.

save_to_excel_name = path + "\\real_components_by_phase_angle.xlsx"
real_components.to_excel(save_to_excel_name)

In [ ]:
# Save numerical results for 6th harmonic.

# Find peak phase angles for each wavenumber.

# Fourier transformed output and phase angles

output_and_angles6 = pd.concat([dft.iloc[6, : ], angles.iloc[6, : ]], axis = 1)
output_and_angles6.columns = ["Discrete Fourier Transform", "Phase Angle"]
save_to_excel_name = path + "\\FT_output_and_angles6.xlsx"
output_and_angles6.to_excel(save_to_excel_name)

#Save real components by phase angle data.

save_to_excel_name = path + "\\real_components_by_phase_angle6.xlsx"
real_components6.to_excel(save_to_excel_name)

In [ ]:
# Filter discrete Fourier transform over the scans in each column/wavenumber, remove undesired frequencies.

evens = np.arange(0, 120, 2)
filtered_dft = dft.copy()

for row in evens:    # Remove even frequencies 
    filtered_dft.iloc[row] = 0
    
filtered_dft.iloc[6:] = 0    # Remove high frequencies 
# filtered_dft.iloc[2:] = 0    # Remove all but fundamental frequency 
filtered_dft.head(10)

In [ ]:
# Perform an inverse discrete Fourier transform over the scans in each column/wavenumber.
# This will return the data from the frequency domain to the time domain (scans).

idft_array = np.real(scipy.fft.ifft(filtered_dft.T))
idft = pd.DataFrame(idft_array, index = dft.columns).T
idft.head()

In [ ]:
# Take the sum of the absolute value of all scans.

y_scale = 0.2

fig, ax = plt.subplots(1, 1, figsize=(25, 5))
ax.plot(idft.columns.astype(int), idft.abs().sum(), "b")
ax.set(
    xlim = (4000, 800),
    ylim = (-y_scale/10, y_scale),
    xticks = np.arange(4000, 750, -100),
    #yticks = np.arange(0, 370, 20)
)
plt.title("DFT, Filtered, and IDFT Cumulative Spectrum")
plt.ylabel("K.M.")
plt.xlabel("Wavenumber $cm^{-1}$")
# plt.savefig(path + "\\Transformed_filtered_spectrum_wide.jpg")    # Uncomment to save
plt.show()

In [ ]:
# Compare Reflectance Plotting Conventions.

x = idft.columns.astype(int)
y = idft.abs().sum()
fig, ax = plt.subplots(1, 1, figsize=(15, 3))
ax.plot(x, y, "b", label = "K.M.")    # K.M.
# ax.plot(x, logR(inv_KM(y)), "r", label = "log(R)")    # log(R)
ax.plot(x, log_inv_R(inv_KM(y)), "g", label = "log(1/R)")    # log(1/R)

# ax.plot(summed_normalized.columns, summed_normalized.iloc[0], "r")
ax.set(
    xlim = (4000, 800),
    ylim = (-y_scale/10, y_scale),
    xticks = np.arange(4000, 750, -200),
    #yticks = np.arange(0, 370, 20)
)
plt.title("Compare Reflectance Plotting Conventions")
plt.ylabel("")
plt.xlabel("Wavenumber $cm^{-1}$")
plt.legend()
# plt.savefig(path + "\\Transformed_filtered_spectrum_wide_both.jpg", dpi = 300)    # Uncomment to save
plt.show()